Connect to GPU

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#! git clone https://github.com/eengel7/comparison_NLP_classification_models.git

In [ ]:
%cd drive/MyDrive/comparison_NLP_classification_models

In [ ]:
! pip install -r requirements.txt

In [ ]:
! git pull

In [1]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: eengel7. Use `wandb login --relogin` to force relogin


True

In [ ]:
import torch
print(torch.cuda.is_available())
import tensorflow as tf
print(tf.test.gpu_device_name())
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [4]:
from src.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)
import pandas as pd
import logging
import pickle
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from src.training import train_model
from src.evaluation import eval_model
from src.preprocessing.get_preprocessed_data import get_preprocessed_data
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier 

from src.utils import prepare_df
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import preprocessing
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Experiment config:----------------------
model_type = 'bert'
model_name = "bert-base-uncased" # 'bert-base-uncased' ,"roberta-base", "outputs/checkpoint-120-epoch-1",

model_args = MultiLabelClassificationArgs(
                                          wandb_project ='multi-label-42', 
                                          wandb_kwargs = {"name": model_name},
                                          learning_rate = 5e-5,
                                          num_train_epochs=10,
                                          train_batch_size = 4,
                                          eval_batch_size = 4,
                                          evaluate_during_training= True, 
                                          use_multiprocessing= True,
                                          use_early_stopping= True,
                                          early_stopping_patience=3,
                                          early_stopping_delta= 1e-5
                                          ) 
model = MultiLabelClassificationModel(
    model_type,
    model_name,
    num_labels=305,
    args=model_args,
    # use_cuda = True
)
# -----------------------------------
# Prepare data
X_train, X_test, X_val, Y_train, Y_test, Y_val = get_preprocessed_data(model_type, overwrite_data= False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification were not 

preprocessed_data.pkl already exists at data/preprocessed/bert_en_all_levels_val.
Data already exists and will not be overwritten.


In [5]:
train_df = prepare_df(X_train, Y_train)
test_df = prepare_df(X_test, Y_test)
val_df = prepare_df(X_val, Y_val)
print('Data is loaded. Training starts.')

train_model(model, train_df, eval_df = val_df)

# # Evaluate the model
# result, model_outputs, wrong_predictions = eval_model(model,
#     test
# )

#print(result, model_outputs, wrong_predictions)
# # Make predictions with the model
# predictions, raw_outputs = model.predict(["Sam"])

# print(predictions,raw_outputs)


INFO:src.classification.classification_utils: Converting to features started. Cache is not used.


Data is loaded. Training starts.


  0%|          | 10/26877 [00:31<23:26:47,  3.14s/it]


KeyboardInterrupt: 

In [2]:
from src.preprocessing.get_preprocessed_data import get_preprocessed_data
import logging

import wandb
from config.data_args import DataArgs
from src.classification import (MultiLabelClassificationArgs,
                                MultiLabelClassificationModel)
from src.evaluation import eval_model
from src.preprocessing.get_preprocessed_data import get_preprocessed_data
from src.training import train_model
from src.utils import prepare_df
import pandas as pd


X_train, X_test, X_val, Y_train, Y_test, Y_val = get_preprocessed_data('bert', overwrite_data= False, random_seed= 42)  


# Initialise the model
model_args = MultiLabelClassificationArgs(
                                        learning_rate = 5e-5,
                                        num_train_epochs=15,
                                        train_batch_size = 4,
                                        use_multiprocessing= False,
                                        use_early_stopping= True,
                                        early_stopping_patience=3,
                                        early_stopping_delta= 0
                                        ) 

model = MultiLabelClassificationModel( 
    'bert',
    'bert-base-uncased',
    num_labels=305,
    use_cuda = False
)
train_df = prepare_df(X_train, Y_train)
val_df = prepare_df(X_val, Y_val)
test_df = prepare_df(X_test, Y_test)

combined_df = pd.concat([train_df, val_df, test_df], axis=0)
# Training
print(combined_df)
print('Training starts.')
train_model(model, combined_df)

preprocessed_data.pkl already exists at data/preprocessed/transformer_en_all_levels_val_42.
Data already exists and will not be overwritten.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification were not 

                                                    text  \
4135   Developing strategies for scientific excellenc...   
34132  Feasibility study for implementation of AI-bas...   
38440  Internet of Things Initiative\nInternet of Thi...   
42733  Advancing cancer detection through metabolism-...   
9764   Sustainable manufacturing of 3D printed sandmo...   
...                                                  ...   
18507  From assistant to associate professor: Career ...   
4346   New Smart Nanoparticle Materials: Development ...   
21120  Participation in the JPI Oceans Pilot Action ´...   
34511  A New Region of the World?\nA New Region of th...   
37371  NeoCel – Novel processes for sustainable cellu...   

                                                  labels  
4135   [1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, ...  
34132  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
38440  [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...  
42733  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,

  0%|          | 48/41997 [02:09<31:24:09,  2.69s/it]


KeyboardInterrupt: 